In [ ]:
#Inference Time 

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import time
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, GRU
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from numpy import loadtxt
from torch.utils.data import TensorDataset, DataLoader
from keras.models import load_model
from time import sleep
from tqdm import tqdm, trange
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim

In [ ]:
class LSTM_TM(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len):
        super(LSTM_TM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #self.n_layer=n_layer
        self.seq_len = seq_len
        #self.pred_len = pred_len
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size= hidden_dim,
            batch_first = True,
            #bidirectional=True,
            dropout=0.2)
        self.fc = nn.Linear(hidden_dim, input_dim)
        #self.time_linear = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        y_pred = x[:,-1]
        return y_pred

In [ ]:
class BiLSTM_TM(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len):
        super(BiLSTM_TM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #self.n_layer=n_layer
        self.seq_len = seq_len
        #self.pred_len = pred_len
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size= hidden_dim,
            batch_first = True,
            bidirectional=True,
            dropout=0.2)
        self.fc = nn.Linear(2*hidden_dim, input_dim)
        #self.time_linear = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        y_pred = x[:,-1]
        return y_pred

In [ ]:
class GRU_TM(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len):
        super(GRU_TM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #self.n_layer=n_layer
        self.seq_len = seq_len
        #self.pred_len = pred_len
        
        self.lstm = nn.GRU(
            input_size=input_dim,
            hidden_size= hidden_dim,
            batch_first = True,
            #bidirectional=True,
            dropout=0.2)
        self.fc = nn.Linear(hidden_dim, input_dim)
        #self.time_linear = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        y_pred = x[:,-1]
        return y_pred

In [ ]:
class BiGRU_TM(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len):
        super(BiGRU_TM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #self.n_layer=n_layer
        self.seq_len = seq_len
        #self.pred_len = pred_len
        
        self.lstm = nn.GRU(
            input_size=input_dim,
            hidden_size= hidden_dim,
            batch_first = True,
            bidirectional=True,
            dropout=0.2)
        self.fc = nn.Linear(2*hidden_dim, input_dim)
        #self.time_linear = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        y_pred = x[:,-1]
        return y_pred

In [ ]:
dataset_path = ('testbed_flat_tms_up.csv')
dataset = pd.read_csv(dataset_path, delimiter=";", parse_dates=["Date"])
print(dataset.shape)

dataset=dataset.set_index(['Date'])
dataset.head()
total_steps = dataset.shape[0]
train_size = int(total_steps * 0.7)
val_size = int(total_steps * 0.1)

train_df, val_df, test_df = dataset[0:train_size], dataset[train_size:train_size + val_size], \
                                dataset[train_size + val_size: ] 

train_df=pd.DataFrame(train_df)
test_df=pd.DataFrame(test_df)
val_df=pd.DataFrame(val_df)

sc = MinMaxScaler(feature_range=(0,1))
sc=sc.fit(train_df)
train_df = sc.transform(train_df)
test_df = sc.transform(test_df)
val_df = sc.transform(val_df)
#print("train_df", train_df)
#print("test_df", test_df)
train_df=pd.DataFrame(train_df) 
test_df=pd.DataFrame(test_df)
val_df=pd.DataFrame(val_df)

EPS = 1e-8


def rse(preds, labels):
    return torch.sum((preds - labels) ** 2) / torch.sum((labels + EPS) ** 2)


def mae(preds, labels):
    return torch.mean(torch.abs(preds - labels))


def mse(preds, labels):
    return torch.mean((preds - labels) ** 2)

def mape(preds, labels):
    return torch.mean(torch.abs((preds - labels) / (labels + EPS)))

def rmse(preds, labels):
    return torch.sqrt(torch.mean((preds - labels) ** 2))


def calc_metrics(preds, labels):
    return rse(preds, labels), mae(preds, labels), mse(preds, labels), mape(preds, labels), rmse(preds, labels)

def create_dataset(X, y , seq_len=1):
    Xs, ys = [], []
    for i in tqdm (range(len(X)-seq_len)):
        v=X.iloc[i:(i+seq_len)].to_numpy()
        Xs.append(v)
        ys.append(y.iloc[i+seq_len])
    return np.array(Xs), np.array(ys)

def testing(model, test_loader, out_seq_len):
    model.eval()
    outputs = []
    y_real = []
    for i, (x, y) in enumerate(test_loader):

        preds = model(x)
      
        preds = preds

        outputs.append(preds)
        y_real.append(y)

    yhat = torch.cat(outputs, dim=0)
    y_real = torch.cat(y_real, dim=0)
    test_met = []

    yhat[yhat < 0.0] = 0.0

    if len(yhat.size()) != len(y_real.size()):
        yhat = torch.reshape(yhat, y_real.shape)

    test_met.append([x.item() for x in calc_metrics(yhat, y_real)])
    test_met_df = pd.DataFrame(test_met, columns=['rse', 'mae', 'mse', 'mape', 'rmse']).rename_axis('t')
    return test_met_df, y_real, yhat

input_dim = 196
hidden_dim = 100
#lossfn = torch.nn.MSELoss()
seq_length=[60]
runs = [1,2,3,4,5]



for seq_len in seq_length:
    print('***seq_len***', seq_len)
    for run in runs:
        print('<--run-->', run)
        model = LSTM_TM(input_dim, hidden_dim, seq_len=seq_len)
        weights = torch.load('Inference_time/logs_pred/BiLSTM_data_sdn_seq_60/run_2/best_model.pth')
        model.load_state_dict(weights)
        #model.to(device)

        #optimizer = optim.Adam(model.parameters(), lr=0.0001)
        x_train, y_train = create_dataset(train_df, train_df, seq_len)
        x_test, y_test = create_dataset(test_df, test_df, seq_len)
        x_val, y_val = create_dataset(val_df, val_df, seq_len)

        train_loader = TensorDataset(torch.from_numpy(x_train).float(),torch.from_numpy(y_train).float())   
        val_loader = TensorDataset(torch.from_numpy(x_val).float(),torch.from_numpy(y_val).float())
        test_loader = TensorDataset(torch.from_numpy(x_test).float(),torch.from_numpy(y_test).float())

        train_loader = DataLoader(train_loader, batch_size=64, shuffle=False)
        val_loader = DataLoader(val_loader, batch_size=64, shuffle=False)
        test_loader = DataLoader(test_loader, batch_size=64, shuffle=False)

        print('|--- Testing ---|')
        tmps_test = time.time()
        with torch.no_grad():
            model.eval()
            test_met_df, y_real, yhat = testing(model, test_loader, out_seq_len=1)
            tmps_test2 = time.time() - tmps_test
            print("Test_Temps = %f" % tmps_test2)
        print('Test metric: ', test_met_df)
 
